In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import pickle
import logging
from typing import List, Dict, Tuple
import itertools

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class XGBoostCareerRecommender:
    def __init__(self, n_estimators=100, max_depth=6, learning_rate=0.1,
                 objective='reg:squarederror', random_state=42):
        """
        Initialize the XGBoost Career Recommender

        Args:
            n_estimators: Number of boosting rounds
            max_depth: Maximum depth of trees
            learning_rate: Learning rate
            objective: XGBoost objective function
            random_state: Random seed for reproducibility
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.objective = objective
        self.random_state = random_state

        self.model = None
        self.scaler = StandardScaler()
        self.course_encoder = LabelEncoder()
        self.feature_names = []

    def extract_advanced_features(self, students_df: pd.DataFrame, courses_df: pd.DataFrame) -> pd.DataFrame:
        """
        Extract comprehensive features for XGBoost training

        Args:
            students_df: DataFrame with student information
            courses_df: DataFrame with course information

        Returns:
            DataFrame with engineered features
        """
        features_list = []

        # Define subject categories
        all_subjects = ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'English', 'Geography', 'History']
        stem_subjects = ['Mathematics', 'Physics', 'Chemistry', 'Biology']
        humanities_subjects = ['English', 'Geography', 'History']

        # Define interest categories
        interest_categories = {
            'technology': ['programming', 'computers', 'innovation', 'AI', 'software', 'data', 'machine learning', 'robotics'],
            'healthcare': ['medicine', 'nursing', 'biology', 'helping', 'health', 'medical', 'care'],
            'business': ['entrepreneurship', 'marketing', 'finance', 'management', 'economics', 'business'],
            'creative': ['art', 'design', 'music', 'writing', 'creative', 'media', 'storytelling'],
            'social': ['teaching', 'counseling', 'social work', 'psychology', 'education', 'helping people'],
            'research': ['research', 'analysis', 'investigation', 'study', 'academic'],
            'communication': ['communication', 'public speaking', 'presentation', 'media', 'journalism']
        }

        for index in students_df.index:
            student = students_df.loc[index]

            if 'subjects' not in student:
                logger.error(f"Error: 'subjects' not in student data for index {index}. Columns: {student.index.tolist()}")
                continue # Skip this student if 'subjects' is missing

            student_subjects = set(student['subjects'].split(', '))
            student_interests = set(student['interests'].split(', '))

            for _, course in courses_df.iterrows():
                course_subjects = set(course['subjects'].split(', '))
                course_skills = set(course['skills'].split(', '))

                features = {}

                # Student ID and Course ID (for reference)
                features['student_id'] = student['student_id']
                features['course_name'] = course['course_name']

                # Basic subject matching features
                for subject in all_subjects:
                    features[f'student_has_{subject.lower()}'] = 1 if subject in student_subjects else 0
                    features[f'course_requires_{subject.lower()}'] = 1 if subject in course_subjects else 0

                # Subject overlap features
                features['subject_overlap_count'] = len(student_subjects.intersection(course_subjects))
                features['subject_overlap_ratio'] = (len(student_subjects.intersection(course_subjects)) /
                                                   len(student_subjects.union(course_subjects)) if student_subjects.union(course_subjects) else 0)

                # Interest-skill alignment features
                features['interest_skill_overlap'] = len(student_interests.intersection(course_skills))
                features['interest_skill_ratio'] = (len(student_interests.intersection(course_skills)) /
                                                   len(student_interests.union(course_skills)) if student_interests.union(course_skills) else 0)

                # Subject category features
                student_stem_count = sum(1 for subj in student_subjects if subj in stem_subjects)
                student_humanities_count = sum(1 for subj in student_subjects if subj in humanities_subjects)

                course_stem_count = sum(1 for subj in course_subjects if subj in stem_subjects)
                course_humanities_count = sum(1 for subj in course_subjects if subj in humanities_subjects)

                features['student_stem_ratio'] = student_stem_count / len(stem_subjects)
                features['student_humanities_ratio'] = student_humanities_count / len(humanities_subjects)
                features['course_stem_ratio'] = course_stem_count / len(stem_subjects)
                features['course_humanities_ratio'] = course_humanities_count / len(humanities_subjects)

                # Alignment between student and course preferences
                features['stem_alignment'] = min(features['student_stem_ratio'], features['course_stem_ratio'])
                features['humanities_alignment'] = min(features['student_humanities_ratio'], features['course_humanities_ratio'])

                # Interest category features
                for category, keywords in interest_categories.items():
                    student_category_score = sum(1 for interest in student_interests
                                               if any(keyword.lower() in interest.lower() for keyword in keywords))
                    course_category_score = sum(1 for skill in course_skills
                                              if any(keyword.lower() in skill.lower() for keyword in keywords))

                    features[f'student_{category}_interest'] = student_category_score
                    features[f'course_{category}_relevance'] = course_category_score
                    features[f'{category}_alignment'] = min(student_category_score, course_category_score)

                # Diversity features
                features['student_subject_diversity'] = len(student_subjects)
                features['student_interest_diversity'] = len(student_interests)
                features['course_subject_breadth'] = len(course_subjects)
                features['course_skill_breadth'] = len(course_skills)

                # Target variable: compatibility score
                subject_score = len(student_subjects.intersection(course_subjects))
                interest_score = len(student_interests.intersection(course_skills))
                total_possible = len(student_subjects) + len(student_interests)

                features['compatibility_score'] = (subject_score + interest_score) / total_possible if total_possible > 0 else 0

                features_list.append(features)

        return pd.DataFrame(features_list)

    def prepare_training_data(self, students_df: pd.DataFrame, courses_df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Prepare training data for XGBoost

        Args:
            students_df: DataFrame with student information
            courses_df: DataFrame with course information

        Returns:
            Tuple of (features, targets)
        """
        logger.info("Extracting advanced features...")

        # Extract features
        features_df = self.extract_advanced_features(students_df, courses_df)

        # Separate features and target
        target_col = 'compatibility_score'
        feature_cols = [col for col in features_df.columns
                       if col not in ['student_id', 'course_name', target_col]]

        X = features_df[feature_cols]
        y = features_df[target_col]

        # Store feature names
        self.feature_names = feature_cols

        # Scale features
        X_scaled = self.scaler.fit_transform(X)

        logger.info(f"Prepared {X_scaled.shape[0]} training samples with {X_scaled.shape[1]} features")

        return X_scaled, y.values

    def train(self, students_df: pd.DataFrame, courses_df: pd.DataFrame,
              test_size=0.2, early_stopping_rounds=20):
        """
        Train the XGBoost model

        Args:
            students_df: DataFrame with student information
            courses_df: DataFrame with course information
            test_size: Fraction of data for testing
            early_stopping_rounds: Early stopping patience
        """
        logger.info("Preparing training data...")

        # Prepare training data
        X, y = self.prepare_training_data(students_df, courses_df)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=self.random_state, stratify=None
        )

        # Initialize XGBoost model
        self.model = xgb.XGBRegressor(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            learning_rate=self.learning_rate,
            objective=self.objective,
            random_state=self.random_state,
            reg_alpha=0.1,  # L1 regularization
            reg_lambda=0.1,  # L2 regularization
            subsample=0.8,   # Subsample ratio
            colsample_bytree=0.8,  # Feature sampling
            eval_metric='rmse'
        )

        # Train model without early stopping callbacks to resolve TypeError
        logger.info("Training XGBoost model...")

        self.model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=False
        )

        # Evaluate model
        train_pred = self.model.predict(X_train)
        test_pred = self.model.predict(X_test)

        train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

        logger.info(f"Training RMSE: {train_rmse:.4f}")
        logger.info(f"Test RMSE: {test_rmse:.4f}")

        # Feature importance
        feature_importance = self.model.feature_importances_
        importance_df = pd.DataFrame({
            'feature': self.feature_names,
            'importance': feature_importance
        }).sort_values('importance', ascending=False)

        logger.info("Top 10 most important features:")
        for _, row in importance_df.head(10).iterrows():
            logger.info(f"  {row['feature']}: {row['importance']:.4f}")

        logger.info("Training completed!")

        return {
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'feature_importance': importance_df
        }

    def predict_for_student(self, student_data: Dict, courses_df: pd.DataFrame, top_k=5) -> List[Tuple[str, float]]:
        """
        Predict career recommendations for a single student

        Args:
            student_data: Dictionary with 'subjects' and 'interests' keys
            courses_df: DataFrame with course information
            top_k: Number of top recommendations to return
        Returns:
            List of (course_name, confidence_score) tuples
        """
        if self.model is None:
            raise ValueError("Model not trained yet!")

        # Create a temporary DataFrame for the student
        temp_student_df = pd.DataFrame([{
            'student_id': 'temp_student',
            'subjects': ', '.join(student_data['subjects']),
            'interests': ', '.join(student_data['interests'])
        }])

        # Extract features for all student-course combinations
        features_df = self.extract_advanced_features(temp_student_df, courses_df)

        # Prepare features for prediction
        feature_cols = [col for col in features_df.columns
                       if col not in ['student_id', 'course_name', 'compatibility_score']]

        X = features_df[feature_cols]
        X_scaled = self.scaler.transform(X)

        # Make predictions
        predictions = self.model.predict(X_scaled)

        # Combine with course names
        recommendations = []
        for i, course_name in enumerate(features_df['course_name']):
            recommendations.append((course_name, float(predictions[i])))

        # Sort by prediction score and return top K
        recommendations.sort(key=lambda x: x[1], reverse=True)

        return recommendations[:top_k]

    def get_feature_importance(self, top_n=20) -> pd.DataFrame:
        """
        Get feature importance from trained model

        Args:
            top_n: Number of top features to return

        Returns:
            DataFrame with feature importance
        """
        if self.model is None:
            raise ValueError("Model not trained yet!")

        importance_df = pd.DataFrame({
            'feature': self.feature_names,
            'importance': self.model.feature_importances_
        }).sort_values('importance', ascending=False)

        return importance_df.head(top_n)

    def save_model(self, filepath: str):
        """Save the trained model and preprocessors"""
        if self.model is None:
            raise ValueError("No model to save!")

        # Save XGBoost model
        self.model.save_model(f"{filepath}_xgboost.json")

        # Save preprocessors and metadata
        with open(f"{filepath}_preprocessors.pkl", 'wb') as f:
            pickle.dump({
                'scaler': self.scaler,
                'course_encoder': self.course_encoder,
                'feature_names': self.feature_names,
                'n_estimators': self.n_estimators,
                'max_depth': self.max_depth,
                'learning_rate': self.learning_rate,
                'objective': self.objective,
                'random_state': self.random_state
            }, f)

        logger.info(f"Model saved to {filepath}")

    def load_model(self, filepath: str):
        """Load a trained model and preprocessors"""
        # Load XGBoost model
        self.model = xgb.XGBRegressor()
        self.model.load_model(f"{filepath}_xgboost.json")

        # Load preprocessors and metadata
        with open(f"{filepath}_preprocessors.pkl", 'rb') as f:
            data = pickle.load(f)
            self.scaler = data['scaler']
            self.course_encoder = data['course_encoder']
            self.feature_names = data['feature_names']
            self.n_estimators = data['n_estimators']
            self.max_depth = data['max_depth']
            self.learning_rate = data['learning_rate']
            self.objective = data['objective']
            self.random_state = data['random_state']

        logger.info(f"Model loaded from {filepath}")

def main():
    students_df = pd.read_csv('./sample_data/student_data.csv', on_bad_lines='skip')
    courses_df = pd.read_csv('./sample_data/Courses.csv')

    print("Columns in students_df:", students_df.columns)
    print("Head of students_df:\n", students_df.head())
    print("Columns in courses_df:", courses_df.columns)

    # Initialize recommender
    recommender = XGBoostCareerRecommender(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        objective='reg:squarederror'
    )

    # Train model
    results = recommender.train(students_df, courses_df)

    # Save model
    recommender.save_model('xgboost_career_model')

    # Display feature importance
    importance_df = recommender.get_feature_importance()
    print("\nTop 15 Most Important Features:")
    print(importance_df.head(15))

    # Test prediction
    test_student = {
        'subjects': ['Mathematics', 'Physics'],
        'interests': ['Artificial Intelligence', 'Machine Learning']
    }

    recommendations = recommender.predict_for_student(test_student, courses_df)

    print("\nRecommendations for test student:")
    for course, confidence in recommendations:
        print(f"  {course}: {confidence:.3f}")

if __name__ == "__main__":
    main()

Columns in students_df: Index(['student_id', 'subjects', 'interests'], dtype='object')
Head of students_df:
   student_id                                subjects  \
0       S001  Mathematics, Physics, Computer Studies   
1       S002             English, Kiswahili, History   
2       S003         Biology, Chemistry, Agriculture   
3       S004    Geography, History, Business Studies   
4       S005          Art and Design, Music, English   

                                    interests  
0           Robotics, Artificial Intelligence  
1                    Creative Writing, Poetry  
2  Environmental Science, Sustainable Farming  
3                   Urban Planning, Economics  
4                 Graphic Design, Songwriting  
Columns in courses_df: Index(['course_id', 'course_name', 'subjects', 'skills'], dtype='object')
Successfully accessed 'subjects' for first row: Mathematics, Physics, Computer Studies

Top 15 Most Important Features:
                       feature  importance
14    

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, mean_squared_error, classification_report
import pickle
import logging
from typing import List, Dict, Tuple
import itertools

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class XGBoostCareerRecommender:
    def __init__(self, n_estimators=100, max_depth=6, learning_rate=0.1,
                 objective='reg:squarederror', random_state=42, classification_threshold=0.5):
        """
        Initialize the XGBoost Career Recommender

        Args:
            n_estimators: Number of boosting rounds
            max_depth: Maximum depth of trees
            learning_rate: Learning rate
            objective: XGBoost objective function
            random_state: Random seed for reproducibility
            classification_threshold: Threshold for converting regression to classification
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.objective = objective
        self.random_state = random_state
        self.classification_threshold = classification_threshold

        self.model = None
        self.scaler = StandardScaler()
        self.course_encoder = LabelEncoder()
        self.feature_names = []

    # ...existing code... (extract_advanced_features and prepare_training_data methods remain the same)

    def extract_advanced_features(self, students_df: pd.DataFrame, courses_df: pd.DataFrame) -> pd.DataFrame:
        """
        Extract comprehensive features for XGBoost training

        Args:
            students_df: DataFrame with student information
            courses_df: DataFrame with course information

        Returns:
            DataFrame with engineered features
        """
        features_list = []

        # Define subject categories
        all_subjects = ['Mathematics', 'Physics', 'Chemistry', 'Biology', 'English', 'Geography', 'History']
        stem_subjects = ['Mathematics', 'Physics', 'Chemistry', 'Biology']
        humanities_subjects = ['English', 'Geography', 'History']

        # Define interest categories
        interest_categories = {
            'technology': ['programming', 'computers', 'innovation', 'AI', 'software', 'data', 'machine learning', 'robotics'],
            'healthcare': ['medicine', 'nursing', 'biology', 'helping', 'health', 'medical', 'care'],
            'business': ['entrepreneurship', 'marketing', 'finance', 'management', 'economics', 'business'],
            'creative': ['art', 'design', 'music', 'writing', 'creative', 'media', 'storytelling'],
            'social': ['teaching', 'counseling', 'social work', 'psychology', 'education', 'helping people'],
            'research': ['research', 'analysis', 'investigation', 'study', 'academic'],
            'communication': ['communication', 'public speaking', 'presentation', 'media', 'journalism']
        }

        for index in students_df.index:
            student = students_df.loc[index]

            if 'subjects' not in student:
                logger.error(f"Error: 'subjects' not in student data for index {index}. Columns: {student.index.tolist()}")
                continue # Skip this student if 'subjects' is missing

            student_subjects = set(student['subjects'].split(', '))
            student_interests = set(student['interests'].split(', '))

            for _, course in courses_df.iterrows():
                course_subjects = set(course['subjects'].split(', '))
                course_skills = set(course['skills'].split(', '))

                features = {}

                # Student ID and Course ID (for reference)
                features['student_id'] = student['student_id']
                features['course_name'] = course['course_name']

                # Basic subject matching features
                for subject in all_subjects:
                    features[f'student_has_{subject.lower()}'] = 1 if subject in student_subjects else 0
                    features[f'course_requires_{subject.lower()}'] = 1 if subject in course_subjects else 0

                # Subject overlap features
                features['subject_overlap_count'] = len(student_subjects.intersection(course_subjects))
                features['subject_overlap_ratio'] = (len(student_subjects.intersection(course_subjects)) /
                                                   len(student_subjects.union(course_subjects)) if student_subjects.union(course_subjects) else 0)

                # Interest-skill alignment features
                features['interest_skill_overlap'] = len(student_interests.intersection(course_skills))
                features['interest_skill_ratio'] = (len(student_interests.intersection(course_skills)) /
                                                   len(student_interests.union(course_skills)) if student_interests.union(course_skills) else 0)

                # Subject category features
                student_stem_count = sum(1 for subj in student_subjects if subj in stem_subjects)
                student_humanities_count = sum(1 for subj in student_subjects if subj in humanities_subjects)

                course_stem_count = sum(1 for subj in course_subjects if subj in stem_subjects)
                course_humanities_count = sum(1 for subj in course_subjects if subj in humanities_subjects)

                features['student_stem_ratio'] = student_stem_count / len(stem_subjects)
                features['student_humanities_ratio'] = student_humanities_count / len(humanities_subjects)
                features['course_stem_ratio'] = course_stem_count / len(stem_subjects)
                features['course_humanities_ratio'] = course_humanities_count / len(humanities_subjects)

                # Alignment between student and course preferences
                features['stem_alignment'] = min(features['student_stem_ratio'], features['course_stem_ratio'])
                features['humanities_alignment'] = min(features['student_humanities_ratio'], features['course_humanities_ratio'])

                # Interest category features
                for category, keywords in interest_categories.items():
                    student_category_score = sum(1 for interest in student_interests
                                               if any(keyword.lower() in interest.lower() for keyword in keywords))
                    course_category_score = sum(1 for skill in course_skills
                                              if any(keyword.lower() in skill.lower() for keyword in keywords))

                    features[f'student_{category}_interest'] = student_category_score
                    features[f'course_{category}_relevance'] = course_category_score
                    features[f'{category}_alignment'] = min(student_category_score, course_category_score)

                # Diversity features
                features['student_subject_diversity'] = len(student_subjects)
                features['student_interest_diversity'] = len(student_interests)
                features['course_subject_breadth'] = len(course_subjects)
                features['course_skill_breadth'] = len(course_skills)

                # Target variable: compatibility score
                subject_score = len(student_subjects.intersection(course_subjects))
                interest_score = len(student_interests.intersection(course_skills))
                total_possible = len(student_subjects) + len(student_interests)

                features['compatibility_score'] = (subject_score + interest_score) / total_possible if total_possible > 0 else 0

                features_list.append(features)

        return pd.DataFrame(features_list)

    def prepare_training_data(self, students_df: pd.DataFrame, courses_df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Prepare training data for XGBoost

        Args:
            students_df: DataFrame with student information
            courses_df: DataFrame with course information

        Returns:
            Tuple of (features, targets)
        """
        logger.info("Extracting advanced features...")

        # Extract features
        features_df = self.extract_advanced_features(students_df, courses_df)

        # Separate features and target
        target_col = 'compatibility_score'
        feature_cols = [col for col in features_df.columns
                       if col not in ['student_id', 'course_name', target_col]]

        X = features_df[feature_cols]
        y = features_df[target_col]

        # Store feature names
        self.feature_names = feature_cols

        # Scale features
        X_scaled = self.scaler.fit_transform(X)

        logger.info(f"Prepared {X_scaled.shape[0]} training samples with {X_scaled.shape[1]} features")

        return X_scaled, y.values

    def evaluate_classification_metrics(self, y_true, y_pred):
        """
        Calculate classification metrics by converting regression predictions to binary classification

        Args:
            y_true: True compatibility scores
            y_pred: Predicted compatibility scores

        Returns:
            Dictionary with classification metrics
        """
        # Convert regression to binary classification
        y_true_binary = (y_true >= self.classification_threshold).astype(int)
        y_pred_binary = (y_pred >= self.classification_threshold).astype(int)

        # Calculate accuracy
        accuracy = accuracy_score(y_true_binary, y_pred_binary)

        # Calculate precision, recall, f1-score
        precision, recall, f1, support = precision_recall_fscore_support(
            y_true_binary, y_pred_binary, average='binary'
        )

        # Calculate macro and weighted averages
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
            y_true_binary, y_pred_binary, average='macro'
        )

        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
            y_true_binary, y_pred_binary, average='weighted'
        )

        # Generate classification report
        class_report = classification_report(
            y_true_binary, y_pred_binary,
            target_names=['Not Recommended', 'Recommended'],
            output_dict=True
        )

        return {
            'accuracy': accuracy,
            'precision_binary': precision,
            'recall_binary': recall,
            'f1_binary': f1,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'f1_macro': f1_macro,
            'precision_weighted': precision_weighted,
            'recall_weighted': recall_weighted,
            'f1_weighted': f1_weighted,
            'classification_report': class_report,
            'threshold_used': self.classification_threshold
        }

    def train(self, students_df: pd.DataFrame, courses_df: pd.DataFrame,
              test_size=0.2, early_stopping_rounds=20):
        """
        Train the XGBoost model with comprehensive evaluation

        Args:
            students_df: DataFrame with student information
            courses_df: DataFrame with course information
            test_size: Fraction of data for testing
            early_stopping_rounds: Early stopping patience
        """
        logger.info("Preparing training data...")

        # Prepare training data
        X, y = self.prepare_training_data(students_df, courses_df)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=self.random_state, stratify=None
        )

        # Initialize XGBoost model
        self.model = xgb.XGBRegressor(
            n_estimators=self.n_estimators,
            max_depth=self.max_depth,
            learning_rate=self.learning_rate,
            objective=self.objective,
            random_state=self.random_state,
            reg_alpha=0.1,  # L1 regularization
            reg_lambda=0.1,  # L2 regularization
            subsample=0.8,   # Subsample ratio
            colsample_bytree=0.8,  # Feature sampling
            eval_metric='rmse'
        )

        # Train model
        logger.info("Training XGBoost model...")

        self.model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=False
        )

        # Evaluate model - Regression metrics
        train_pred = self.model.predict(X_train)
        test_pred = self.model.predict(X_test)

        train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

        logger.info(f"Training RMSE: {train_rmse:.4f}")
        logger.info(f"Test RMSE: {test_rmse:.4f}")

        # Evaluate model - Classification metrics
        train_classification_metrics = self.evaluate_classification_metrics(y_train, train_pred)
        test_classification_metrics = self.evaluate_classification_metrics(y_test, test_pred)

        logger.info("\n=== TRAINING SET CLASSIFICATION METRICS ===")
        logger.info(f"Accuracy: {train_classification_metrics['accuracy']:.4f}")
        logger.info(f"Precision (binary): {train_classification_metrics['precision_binary']:.4f}")
        logger.info(f"Recall (binary): {train_classification_metrics['recall_binary']:.4f}")
        logger.info(f"F1-Score (binary): {train_classification_metrics['f1_binary']:.4f}")
        logger.info(f"F1-Score (macro): {train_classification_metrics['f1_macro']:.4f}")
        logger.info(f"F1-Score (weighted): {train_classification_metrics['f1_weighted']:.4f}")

        logger.info("\n=== TEST SET CLASSIFICATION METRICS ===")
        logger.info(f"Accuracy: {test_classification_metrics['accuracy']:.4f}")
        logger.info(f"Precision (binary): {test_classification_metrics['precision_binary']:.4f}")
        logger.info(f"Recall (binary): {test_classification_metrics['recall_binary']:.4f}")
        logger.info(f"F1-Score (binary): {test_classification_metrics['f1_binary']:.4f}")
        logger.info(f"F1-Score (macro): {test_classification_metrics['f1_macro']:.4f}")
        logger.info(f"F1-Score (weighted): {test_classification_metrics['f1_weighted']:.4f}")

        # Feature importance
        feature_importance = self.model.feature_importances_
        importance_df = pd.DataFrame({
            'feature': self.feature_names,
            'importance': feature_importance
        }).sort_values('importance', ascending=False)

        logger.info("\nTop 10 most important features:")
        for _, row in importance_df.head(10).iterrows():
            logger.info(f"  {row['feature']}: {row['importance']:.4f}")

        logger.info("Training completed!")

        return {
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'train_classification_metrics': train_classification_metrics,
            'test_classification_metrics': test_classification_metrics,
            'feature_importance': importance_df
        }

    def detailed_classification_report(self, y_true, y_pred):
        """
        Generate a detailed classification report

        Args:
            y_true: True compatibility scores
            y_pred: Predicted compatibility scores

        Returns:
            Formatted classification report string
        """
        y_true_binary = (y_true >= self.classification_threshold).astype(int)
        y_pred_binary = (y_pred >= self.classification_threshold).astype(int)

        report = classification_report(
            y_true_binary, y_pred_binary,
            target_names=['Not Recommended', 'Recommended']
        )

        return report

    def predict_for_student(self, student_data: Dict, courses_df: pd.DataFrame, top_k=5) -> List[Tuple[str, float]]:
        """
        Predict career recommendations for a single student

        Args:
            student_data: Dictionary with 'subjects' and 'interests' keys
            courses_df: DataFrame with course information
            top_k: Number of top recommendations to return
        Returns:
            List of (course_name, confidence_score) tuples
        """
        if self.model is None:
            raise ValueError("Model not trained yet!")

        # Create a temporary DataFrame for the student
        temp_student_df = pd.DataFrame([{
            'student_id': 'temp_student',
            'subjects': ', '.join(student_data['subjects']),
            'interests': ', '.join(student_data['interests'])
        }])

        # Extract features for all student-course combinations
        features_df = self.extract_advanced_features(temp_student_df, courses_df)

        # Prepare features for prediction
        feature_cols = [col for col in features_df.columns
                       if col not in ['student_id', 'course_name', 'compatibility_score']]

        X = features_df[feature_cols]
        X_scaled = self.scaler.transform(X)

        # Make predictions
        predictions = self.model.predict(X_scaled)

        # Combine with course names
        recommendations = []
        for i, course_name in enumerate(features_df['course_name']):
            recommendations.append((course_name, float(predictions[i])))

        # Sort by prediction score and return top K
        recommendations.sort(key=lambda x: x[1], reverse=True)

        return recommendations[:top_k]

    def get_feature_importance(self, top_n=20) -> pd.DataFrame:
        """
        Get feature importance from trained model

        Args:
            top_n: Number of top features to return

        Returns:
            DataFrame with feature importance
        """
        if self.model is None:
            raise ValueError("Model not trained yet!")

        importance_df = pd.DataFrame({
            'feature': self.feature_names,
            'importance': self.model.feature_importances_
        }).sort_values('importance', ascending=False)

        return importance_df.head(top_n)

    def save_model(self, filepath: str):
        """Save the trained model and preprocessors"""
        if self.model is None:
            raise ValueError("No model to save!")

        # Save XGBoost model
        self.model.save_model(f"{filepath}_xgboost.json")

        # Save preprocessors and metadata
        with open(f"{filepath}_preprocessors.pkl", 'wb') as f:
            pickle.dump({
                'scaler': self.scaler,
                'course_encoder': self.course_encoder,
                'feature_names': self.feature_names,
                'n_estimators': self.n_estimators,
                'max_depth': self.max_depth,
                'learning_rate': self.learning_rate,
                'objective': self.objective,
                'random_state': self.random_state,
                'classification_threshold': self.classification_threshold
            }, f)

        logger.info(f"Model saved to {filepath}")

    def load_model(self, filepath: str):
        """Load a trained model and preprocessors"""
        # Load XGBoost model
        self.model = xgb.XGBRegressor()
        self.model.load_model(f"{filepath}_xgboost.json")

        # Load preprocessors and metadata
        with open(f"{filepath}_preprocessors.pkl", 'rb') as f:
            data = pickle.load(f)
            self.scaler = data['scaler']
            self.course_encoder = data['course_encoder']
            self.feature_names = data['feature_names']
            self.n_estimators = data['n_estimators']
            self.max_depth = data['max_depth']
            self.learning_rate = data['learning_rate']
            self.objective = data['objective']
            self.random_state = data['random_state']
            self.classification_threshold = data.get('classification_threshold', 0.5)

        logger.info(f"Model loaded from {filepath}")

# Example usage and training script
def main():
    # Load data
    students_df = pd.read_csv('./sample_data/student_data.csv', on_bad_lines='skip')
    courses_df = pd.read_csv('./sample_data/Courses.csv')

    print("Columns in students_df:", students_df.columns)
    print("Head of students_df:\n", students_df.head())
    print("Columns in courses_df:", courses_df.columns)

    # Debugging: Access 'subjects' column for the first row
    if not students_df.empty:
        try:
            first_subject = students_df.loc[0, 'subjects']
            print(f"Successfully accessed 'subjects' for first row: {first_subject}")
        except KeyError as e:
            print(f"KeyError when accessing 'subjects' for first row: {e}")
        except Exception as e:
            print(f"An unexpected error occurred when accessing 'subjects' for first row: {e}")

    # Initialize recommender with classification threshold
    recommender = XGBoostCareerRecommender(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        objective='reg:squarederror',
        classification_threshold=0.5  # Adjust this threshold as needed
    )

    # Train model
    results = recommender.train(students_df, courses_df)

    # Save model
    recommender.save_model('xgboost_career_model')

    # Display detailed results
    print("\n" + "="*50)
    print("COMPREHENSIVE MODEL EVALUATION RESULTS")
    print("="*50)

    print("\nRegression Metrics:")
    print(f"Training RMSE: {results['train_rmse']:.4f}")
    print(f"Test RMSE: {results['test_rmse']:.4f}")

    print("\nClassification Metrics (Test Set):")
    test_metrics = results['test_classification_metrics']
    print(f"Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"Precision: {test_metrics['precision_binary']:.4f}")
    print(f"Recall: {test_metrics['recall_binary']:.4f}")
    print(f"F1-Score: {test_metrics['f1_binary']:.4f}")

    print("\nDetailed Classification Report:")
    # Create dummy data for demonstration of the report function
    X, y = recommender.prepare_training_data(students_df, courses_df)
    _, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    test_pred = recommender.model.predict(X_test)
    print(recommender.detailed_classification_report(y_test, test_pred))

    # Display feature importance
    importance_df = recommender.get_feature_importance()
    print("\nTop 15 Most Important Features:")
    print(importance_df.head(15))

    # Test prediction
    test_student = {
        'subjects': ['Mathematics', 'Physics'],
        'interests': ['Artificial Intelligence', 'Machine Learning']
    }

    recommendations = recommender.predict_for_student(test_student, courses_df)

    print("\nRecommendations for test student:")
    for course, confidence in recommendations:
        print(f"  {course}: {confidence:.3f}")

if __name__ == "__main__":
    main()

Columns in students_df: Index(['student_id', 'subjects', 'interests'], dtype='object')
Head of students_df:
   student_id                                subjects  \
0       S001  Mathematics, Physics, Computer Studies   
1       S002             English, Kiswahili, History   
2       S003         Biology, Chemistry, Agriculture   
3       S004    Geography, History, Business Studies   
4       S005          Art and Design, Music, English   

                                    interests  
0           Robotics, Artificial Intelligence  
1                    Creative Writing, Poetry  
2  Environmental Science, Sustainable Farming  
3                   Urban Planning, Economics  
4                 Graphic Design, Songwriting  
Columns in courses_df: Index(['course_id', 'course_name', 'subjects', 'skills'], dtype='object')
Successfully accessed 'subjects' for first row: Mathematics, Physics, Computer Studies

COMPREHENSIVE MODEL EVALUATION RESULTS

Regression Metrics:
Training RMSE: 0.0005